In [17]:
import os
import pandas as pd
import sqlite3
import glob
import os
import logging

db_path_1 = r'C:\Users\trieu.pham\Dropbox\Projects\database\FCdata.db'
db_path_2 = r'C:\Users\PC\Dropbox\Projects\database\FCdata.db'
db_path_3 = '/Users/trieupham/Dropbox/Projects/database/FCdata.db' # Mac

directory_1 = r'C:\Users\trieu.pham\Dropbox\Projects\data_xls\detailTrans'
directory_2 = r'C:\Users\PC\Dropbox\Projects\data_xls\detailTrans'
directory_3 = '/Users/trieupham/Dropbox/Projects/data_xls/detailTrans' # Mac

# Define directories where your .xlsx files are located
directory_sales_by_category_1 = r'C:\Users\trieu.pham\Dropbox\Projects\data_xls\salebycategory'
directory_sales_by_category_2 = r'C:\Users\PC\Dropbox\Projects\data_xls\salebycategory'
directory_sales_by_category_3 = '/Users/trieupham/Dropbox/Projects/data_xls/salebycategory' # Mac

directory_sales_by_store_1 = r'C:\Users\trieu.pham\Dropbox\Projects\data_xls\salebystore'
directory_sales_by_store_2 = r'C:\Users\PC\Dropbox\Projects\data_xls\salebystore'
directory_sales_by_store_3 = '/Users/trieupham/Dropbox/Projects/data_xls/salebystore' # Mac

processed_files_log_path_1 = r'C:\Users\trieu.pham\Dropbox\Projects\github\Python\Python\Data Analysis\Store\log'
processed_files_log_path_2 = r'C:\Users\PC\Dropbox\Projects\github\Python\Python\Data Analysis\Store\log'
processed_files_log_path_3 = '/Users/trieupham/Dropbox/Projects/github/Python/Python/Data Analysis/Store/log' # Mac

import os
# Check if db_path exists
if os.path.exists(db_path_1):
    db_path = db_path_1
    directory = directory_1
    print('Database path: ', db_path)
else:
    if(os.path.exists(db_path_3)):
        db_path = db_path_3
        directory = directory_3
        print('Database path: ', db_path)
    if(os.path.exists(db_path_2)):
        db_path = db_path_2
        directory = directory_2
        print('Database path: ', db_path)
# Check if directory_sales_by_category exists
if os.path.exists(directory_sales_by_category_1):
    directory_sales_by_category = directory_sales_by_category_1
else:
    if(os.path.exists(directory_sales_by_category_3)):
        directory_sales_by_category = directory_sales_by_category_3
    if(os.path.exists(directory_sales_by_category_2)):
        directory_sales_by_category = directory_sales_by_category_2

# Check if directory_sales_by_store exists
if os.path.exists(directory_sales_by_store_1):
    directory_sales_by_store = directory_sales_by_store_1
else:
    if(os.path.exists(directory_sales_by_store_3)):
        directory_sales_by_store = directory_sales_by_store_3
    if(os.path.exists(directory_sales_by_store_2)):
        directory_sales_by_store = directory_sales_by_store_2

# Check if processed_files_log_path exists
if os.path.exists(processed_files_log_path_1):
    processed_files_log_path = processed_files_log_path_1
else:
    if(os.path.exists(processed_files_log_path_3)):
        processed_files_log_path = processed_files_log_path_3
    if(os.path.exists(processed_files_log_path_2)):
        processed_files_log_path = processed_files_log_path_2


Database path:  C:\Users\PC\Dropbox\Projects\database\FCdata.db


In [18]:
# Setup basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# List all .xlsx files in the directory
xlsx_files = glob.glob(os.path.join(directory, '*.xlsx'))

# Exclude files that have been processed in log file
processed_files = []
if os.path.exists(os.path.join(processed_files_log_path, 'processed_files.log')):
    with open(os.path.join(processed_files_log_path, 'processed_files.log'), 'r') as f:
        processed_files = f.read().splitlines()

# Ensure processed_files only contains base file names
processed_files = [os.path.basename(file) for file in processed_files]

# Files which are in xlsx_files but not in processed_files
files_to_process = [file for file in xlsx_files if os.path.basename(file) not in processed_files]

# Output the files to process
files_to_process


['C:\\Users\\PC\\Dropbox\\Projects\\data_xls\\detailTrans\\detailTransExport2024-08-02__3d10a666-2d71-4673-b49a-724f3c31d742.xlsx']

In [19]:
# Define a function to create missing columns in the table
def add_missing_columns(conn, df):
    existing_columns = [row[1] for row in conn.execute("PRAGMA table_info(detail_trans);")]
    new_columns = [col for col in df.columns if col not in existing_columns]
    
    for col in new_columns:
        conn.execute(f'ALTER TABLE detail_trans ADD COLUMN "{col}" FLOAT;')
        logging.info(f'Added new column: {col}')

# Define a function to rename specific columns
def rename_columns(df):
    rename_map = {
        'Ví VNPAY': 'VNPAY',
        'Tiền VND': 'VND',
        'Ví MOMO': 'MOMO',
        'Ví ZaloPay': 'ZaloPay',
        'Grab Order': 'GrabOrder'
    }
    df.rename(columns=rename_map, inplace=True)
    return df

# Use a context manager to ensure the database connection is closed properly
with sqlite3.connect(db_path) as conn:
    for file in files_to_process:
        try:
            # Load the data from the .xlsx file into a DataFrame
            df = pd.read_excel(file)
            
            # Rename specific columns to match the database schema
            df = rename_columns(df)
            
            # Add any missing columns to the detail_trans table
            add_missing_columns(conn, df)
            
            # Insert the data into the detail_trans table
            df.to_sql('detail_trans', conn, if_exists='append', index=False)
            
            logging.info(f'Successfully inserted data from {file}')
        # Record file run to log file
            with open(os.path.join(processed_files_log_path, 'processed_files.log'), 'a') as f:
                f.write(f'{file}\n')
        except Exception as e:
            logging.error(f'Error inserting data from {file}: {e}')
            # Optionally, continue to the next file or handle the error as needed

# No need to explicitly commit or close the connection, as the context manager handles it
logging.info('All files have been processed.')

2024-08-03 18:56:08,414 - INFO - Successfully inserted data from C:\Users\PC\Dropbox\Projects\data_xls\detailTrans\detailTransExport2024-06-20__6aa76d11-a63b-4988-9ab4-c6899d8947b0.xlsx
2024-08-03 18:56:08,512 - INFO - Successfully inserted data from C:\Users\PC\Dropbox\Projects\data_xls\detailTrans\detailTransExport2024-06-21__c7814487-c61f-43ab-a183-c850ceeaae22.xlsx
2024-08-03 18:56:08,627 - INFO - Successfully inserted data from C:\Users\PC\Dropbox\Projects\data_xls\detailTrans\detailTransExport2024-06-22__a91cca7c-d648-4134-ade0-05ba3971f8f1.xlsx
2024-08-03 18:56:08,716 - INFO - Successfully inserted data from C:\Users\PC\Dropbox\Projects\data_xls\detailTrans\detailTransExport2024-06-23__0909e2a5-b6d6-4c49-89bc-ff803f419cc5.xlsx
2024-08-03 18:56:08,822 - INFO - Successfully inserted data from C:\Users\PC\Dropbox\Projects\data_xls\detailTrans\detailTransExport2024-06-24__e3ff305d-2668-44f9-a58e-c9f58f47c780.xlsx
2024-08-03 18:56:08,911 - INFO - Successfully inserted data from C:\Us